In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import shap
import pandas_profiling
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import folium
from folium import plugins
%matplotlib inline

In [0]:
# Predicting the average SAT score per School

In [0]:
df = pd.read_csv("use_this.csv", index_col=0)
df = df.rename(index=str, columns={'AY12-13':'budget_13', 'AY13-14': 'budget_14', 'Economic Need Index':'eni', 'Percent Asian':'asian', 'Percent White':'white', 
                               'Percent Hispanic':'hispanic', 'Percent Black':'black', 'Student Attendance Rate':'attendance',
                               'Percent of Students Chronically Absent':'chronical_absence',
                               'Supportive Environment - Percent Positive':'supportive_env',
                               'Percent English Language Learners':'english_learner', 'Percent Students with Disabilities':'disability',
                               'AvgOfNoCrim N':'avg_num_crime',
                               'Total Grads':'num_grads', 'Dropout':'num_dropout', 'Still Enrolled':"num_cont_enrolled",
                               'Average Score (SAT Math)':"avg_sat_math",
                               'Average Score (SAT Reading)':"avg_sat_reading",
                               'Average Score (SAT Writing)':"avg_sat_writing",
                               'Percent Tested':"percent_tested"})
df = df.dropna()
print(len(df))

1. Data Cleaning
We converted certain columns with string percentage values to floats

In [0]:
def convert_percent(string):
    return float(string.strip('%'))/100
to_convert = ['eni', 'asian', 'white', 'hispanic', 'black',
              'attendance', 'chronical_absence',  'supportive_env',
              'english_learner', 'disability', 'percent_tested']
for column in to_convert:
    df[column] = df[column].apply(convert_percent)

Graduation rates


In [0]:
df['grad rate'] = df['num_grads']/(df['num_grads'] + df['num_dropout'] + df['num_cont_enrolled'])

In [0]:
df.dtypes

Focused correlation matrix¶

In [0]:
columns = ['budget_13', 'budget_14', 'eni', 'asian', 'white', 'hispanic', 'black','grad rate', 'attendance'] 
corr = df[columns].corr()
Col = df.columns.values
fig, ax =plt.subplots(figsize=(9, 9))
sns.heatmap(corr, xticklabels=columns, yticklabels=columns, annot = True, annot_kws={'size':15}, ax = ax)
heat_map=plt.gcf()

Data whitening/standardization¶

In [0]:
df_new = df.loc[df['grad rate'] <= 0.75]
df_new.head()

In [0]:
X_train = df_new[['budget_14', 'eni','attendance']]
y = df_new['grad rate']
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X_train)

Linear Regression (without changing ENI)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
linear_r = LinearRegression()
linear_r.fit(X_train, y_train)
predictions = linear_r.predict(X_test)
print(np.average(predictions))

In [0]:
print(metrics.mean_squared_error(y_test, predictions))
print(metrics.median_absolute_error(y_test, predictions))

Linear Regression¶

In [0]:
df_hope = df.loc[df['grad rate'] <= 0.75]
X_train_hope = df_hope[['budget_14', 'eni','attendance']]
y_train_hope = df_hope['grad rate']
X_train_hope['eni'] = X_train_hope['eni'] - 0.2
y_train_hope.head()
scaler_X_hope = StandardScaler()
X_scaled_hope = scaler_X_hope.fit_transform(X_train_hope)

In [0]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_scaled_hope, y_train_hope, test_size=0.33, random_state=42)
linear_r_hope = LinearRegression()
linear_r_hope.fit(X_train_new, y_train_new)
new_predictions = linear_r.predict(X_test_new)
print(np.average(predictions))